# Multi-messenger astronomy in the context of the VO framework.

### Partecipant project for the [second ESCAPE Virtual Observatory School](https://indico.in2p3.fr/event/25225/): a practical example using  the Multi Order Coverage maps in Python.
##### Maria Lisa Brozzetti; marialisa.brozzetti@studenti.unipg.it
##### Updated by Gianmarco Aristei and Andrea Svizzeretto (gianmarco.aristei@studenti.unipg.it; andrea.svizzeretto@studenti.unipg.it)

The tutorial aims to analyze the [first multi-messenger event, GW170817/GRB170817/AT2017gfo](https://www.ligo.org/science/Publication-GW170817MMA/index.php) using the Multi Order Coverage map (MOC) to encode the gravitational-wave sky localizations from the LIGO and Virgo collaborations, and the gamma-ray-burst error boxes provided by the GBM instrument on board the Fermi Gamma-Ray Space Telescope and calculate from the IPN triangulation using the delay between Fermi and INTEGRAL. 

In [ ]:
!pip3 install mocpy

In [ ]:
!pip3 install astropy

In [ ]:
!pip3 install healpy

In [ ]:
!pip3 install ipyaladin

In [ ]:
#!conda install -c conda-forge ligo.skymap

In [ ]:
import matplotlib.pyplot as plt

import mocpy
from mocpy import MOC, STMOC, WCS

import astropy
from astropy.coordinates import SkyCoord, Angle, Longitude, Latitude

from astropy.time import Time
import astropy.units as u

import healpy as hp

## 1. Encoding of 2D credible regions in a MOC map


## 1.1 Download the sky maps


In [ ]:
# Initial sky localization with the two LIGO interferometers (L1+H1).
!curl -O https://dcc.ligo.org/public/0146/G1701985/001/bayestar_no_virgo.fits.gz

# Initial sky localization with the two LIGO + Virgo (L1+H1+V1).
!curl -O https://dcc.ligo.org/public/0146/G1701985/001/bayestar.fits.gz
    
# Final sky localization published in the (GWTC-1).
!curl -O https://dcc.ligo.org/public/0157/P1800381/007/GW170817_skymap.fits.gz
    
### Fermi/GBM localization of GRB 170817A
!curl -O https://gammaray.nsstc.nasa.gov/gbm/science/grbs/grb170817a/gbuts_healpix_systematic.fit  

## 1.2 Create credible regions encoded with MOC

From [ligo.skymap](https://lscsoft.docs.ligo.org/ligo.skymap/), it is possible to invoke the command-line [ligo-skymap-contour-moc](https://lscsoft.docs.ligo.org/ligo.skymap/tool/ligo_skymap_contour_moc.html). The method returns a credible level encoded in a MOC data structure of an all-sky probability map in FITS format. The method is based on [mocpy](https://cds-astro.github.io/mocpy/). 

Recently, the [mocpy version 0.10.0](https://cds-astro.github.io/mocpy/) builds the credibility regions with an algorithm that has various options, including one which produces the same results as
Aladin Desktop (see a tutorial [here](https://indico.in2p3.fr/event/25225/sessions/15780/attachments/69008/97180/MMA_Aladin_v1a_comp.pdf)), and others that allow for different handling of the
region boundaries. The method is [from_multiordermap_fits_file](https://cds-astro.github.io/mocpy/stubs/mocpy.MOC.html#mocpy.MOC.from_multiordermap_fits_file) method. This method digests only sky maps in the [multi-order format](https://emfollow.docs.ligo.org/userguide/tutorial/multiorder_skymaps.html) (in
which different HEALPix orders are included). To convert a standard HEALPix
format (where all the cells are at a single order) into a multi-order format, the command-line [ligo-skymap-unflatten](https://lscsoft.docs.ligo.org/ligo.skymap/tool/ligo_skymap_unflatten.html) can be applied.

In [ ]:
# Credible region (90%) with the two LIGO interferometers (L1+H1).
!ligo-skymap-contour-moc  'bayestar_no_virgo.fits.gz' -c 90  --output 'credible_region_2ifos'

# Credible region (90%) with the two LIGO + Virgo (L1+H1+V1).
!ligo-skymap-contour-moc  'bayestar.fits.gz' -c 90  --output 'credible_region_3ifos'

# Credible region (90%) - final sky localization published in the (GWTC-1).
!ligo-skymap-contour-moc  'GW170817_skymap.fits.gz' -c 90  --output 'credible_region_final'

# Credible region (90%) Fermi/GBM localization of GRB 170817A
!ligo-skymap-contour-moc  'gbuts_healpix_systematic.fit' -c 90  --output 'fermi_gbm'

### 1.2.1 IPN triangulation

TITLE:   GCN CIRCULAR

NUMBER:  21515

SUBJECT: LIGO/Virgo G298048: IPN triangulation of Fermi/GBM trigger 524666471/170817529

DATE:    17/08/17 18:35:12 GMT

FROM:    Dmitry Svinkin at Ioffe Institute  <svinkin@mail.ioffe.ru>

D. Svinkin, K. Hurley, on behalf of the IPN,

A. von Kienlin, X. Zhang, A. Rau, V. Savchenko, E. Bozzo,
and C. Ferrigno, on behalf of the INTEGRAL SPI-ACS GRB team,

V. Connaughton, M. S. Briggs, C. Meegan, V. Pelassa,
and A. Goldstein, on behalf of the Fermi GBM team, report:

A short-duration burst (Connaughton, LVC GCN Circ. 21506)
has been detected by Fermi (GBM; trigger 524666471),
and INTEGRAL (SPI-ACS), so far, at about 45666 s UT (12:41:06).

We have triangulated it to a GBM-INTEGRAL annulus centered at 
**RA(2000)=119.131deg** (07h 56m 31s)  **Dec(2000)=-44.133deg** (-44d 07' 
57"), whose radius is 73.554 +/- 19.687 deg (3 sigma). The annulus is 
consistent with the Fermi-GBM ground position 
(glg_loclist_all_bn170817529_v01.txt).

This annulus may be improved.

A triangulation map is posted at
http://www.ioffe.ru/LEA/GRBs/GRB170817_T45666/IPN/

In [ ]:
# Creating the MOC annulus provided by IPN.
ipn = MOC.from_ring(lon=Longitude(119.131 * u.deg),lat=Latitude(-44.133 * u.deg),
                         internal_radius=Angle(53.867, u.deg),
                         external_radius=Angle(93.241, u.deg),
                         max_depth=10)

# Save in fits format.
ipn.write("ipn_triangulation", overwrite=True)

## 2. Plotting using Matplotlib

In [ ]:
fig = plt.figure(111, figsize=(10, 10))

with WCS(fig, fov=170 * u.deg,
                     center=SkyCoord(170.0, -40.0, 
                                     unit='deg', frame='icrs'),
                     coordsys="icrs",
                     rotation=Angle(0, u.degree),
                     projection='SIN') as wcs:
    ax = fig.add_subplot(1, 1, 1, projection=wcs)
    
    # Loading MOC IPN annulus
    ipn_triangulation = MOC.load("ipn_triangulation")
    
    # Plot IPN annulus
    ipn_triangulation.fill(ax=ax, wcs=wcs, alpha=0.7,linewidth=0, fill=True, 
                           color='lightblue', 
                           label='The IPN triangulation')
    ipn_triangulation.border(ax=ax, wcs=wcs, alpha=0.7, color="lightblue")

    # Load Fermi/GBM error box of GRB 170817
    fermi_gbm = MOC.load("fermi_gbm")
    
    # Plot Fermi/GBM error box of GRB 170817
    fermi_gbm.fill(ax=ax, wcs=wcs, alpha=0.9,linewidth=0, fill=True, 
                   color="#74bbfb", 
                   label='Fermi/GBM localization')
    fermi_gbm.border(ax=ax, wcs=wcs, alpha=0.9, color="#74bbfb")
        
    # Loading L1+H1 MOC credible region
    credible_region_2ifos = MOC.load("credible_region_2ifos")
    
    # Plot L1+H1 credible region
    credible_region_2ifos.fill(ax=ax, wcs=wcs, alpha=0.5,linewidth=0, fill=True, 
                               color='green', 
                               label='90% credible region with H1+L1')
    credible_region_2ifos.border(ax=ax, wcs=wcs, alpha=0.5, color="green")
    
    # Loading L1+H1+V1 MOC credible region
    credible_region_3ifos = MOC.load("credible_region_3ifos")
    
    # Plot L1+H1+V1 credible region
    credible_region_3ifos.fill(ax=ax, wcs=wcs, alpha=0.7,linewidth=0, fill=True, 
                               color="red", 
                               label='90% credible region with H1+L1+V1')
    credible_region_3ifos.border(ax=ax, wcs=wcs, alpha=0.8, color="red")
             
    # Loading MOC credible region (90%) (GWTC-1).
    credible_region_final = MOC.load("credible_region_final")
    
    # Plot credible region (90%) - published in the (GWTC-1).
    credible_region_final.fill(ax=ax, wcs=wcs, alpha=0.7,linewidth=0, 
                               fill=True, color="orange", 
                               label='90% credible region - GWTC-1')
    credible_region_final.border(ax=ax, wcs=wcs, alpha=0.7, color="orange")
    
#    print('The intersection sky area is {:.2f} sq. deg.'.format(intersection_area))
    
    ax.legend()
    plt.title('Localisation of the gravitational-wave and gamma-ray signals')
    
    plt.grid(color="black", linestyle="dotted")
    plt.show()

## 3. Areas and disjoint regions

In [ ]:
area_2ifo = round(credible_region_2ifos.sky_fraction*41253,2)
print("The Area (90%) with L1 and H1: " + str(area_2ifo) +'°^2' )
print("The number of the disjoint regions: " + 
      str(credible_region_2ifos.split_count()))
print()

area_3ifo = round(credible_region_3ifos.sky_fraction*41253,2)
print("The Area (90%) with L1, H1 and V1: " + str(area_3ifo) +'°^2' )
print("The number of the disjoint regions: " + 
      str(credible_region_3ifos.split_count()))
print()

area_final = round(credible_region_final.sky_fraction*41253,2)
print("The Area (90%) in the final analysis: " + str(area_final) +'°^2' )
print("The number of the disjoint regions: " + 
      str(credible_region_final.split_count()))

In [ ]:
credible_region_2ifos.split_count()

## 4. Interactive visualization using ipyladin widgets

In [ ]:
from ipyaladin import Aladin
import ipyaladin as ipyal

aladin = ipyal.Aladin(target='NGC 4993', fov=180, survey ='P/DSS2/color')
aladin

Scroll up and visualize the credible region contours in the interactive Aladin widgets. From the `Manage Layers` <img src="https://github.com/ggreco77/Tutotest/blob/main/images/ipyaladin_layer.png?raw=true" alt="the Layer Button" style="width:30px; display: inline-block;"/> the credible regions can be independently selected. 

If you hover the mouse pointer over a credible region plan a message will be displayed above it to show the **coverage in sky percentage**.

In [ ]:
# Show the two credible regions in the Aladin widget with 2 different colors.
colors = [ "lightblue", "#74bbfb", "green", "red", "orange"]

credible_regions = ["ipn_triangulation", "fermi_gbm", "credible_region_2ifos",
                   "credible_region_3ifos", "credible_region_final"]

for credible_region, color in zip(credible_regions, colors):
    aladin.add_moc_from_URL(credible_region, {'color': color, 'opacity': 0.7,
                            'adaptativeDisplay': False, 'name':credible_region})

## 4.1 Gravitational-wave sky localization and GRB error box

In [ ]:
from ipywidgets import Layout, Box, widgets

# Initialize the boxes
left_box = Aladin(layout=Layout(width='50%'), target='NGC 4993', fov=180,survey='P/DSS2/color')
right_box = Aladin(layout=Layout(width='50%'))

# Synchronize target between 3 widgets
widgets.jslink((left_box, 'target'), (right_box, 'target'))
# Synchronize FoV (zoom level) between 2 widgets
widgets.jslink((left_box, 'fov'), (right_box, 'fov'))

items = [left_box,right_box]

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
box = Box(children=items, layout=box_layout)
box

In [ ]:
left_box.add_moc_from_URL('fermi_gbm' ,{'color': '#74bbfb' , 'opacity': 0.7,
                            'adaptativeDisplay': False, 'name': 'Fermi/GBM error box of GRB 170817' })

right_box.add_moc_from_URL('credible_region_final', {'color': 'orange', 'opacity': 0.7, 
                                      'adaptativeDisplay': False, 'name': 'Sky localization of GW170817' })

## 5. Spatial and Temporal intersection
**Two astrophysics events are considered coincident if they are within a particular time window of each other**.


Following the [LIGO-Virgo Pubic Alert User Guide](https://emfollow.docs.ligo.org/userguide/index.html) in the section [Coincident with External Trigger Search](https://emfollow.docs.ligo.org/userguide/analysis/searches.html#coincident-with-external-trigger-search), we use the time window from -1 s to 5 s around the GW time. It means that we take into account GRBs up to one
second before or up to 5 seconds after the GW time. Following this prescription, the temporal interval to define the Space and Time MOC ranges from **2017-08-17T12:41:03 UTC to 2017-08-17T12:41:09 UTC**.

In the case of the GRB 170817, we added the trigger time reported in [An Ordinary Short Gamma-Ray Burst with
Extraordinary Implications: Fermi-GBM Detection of GRB 170817A](https://iopscience.iop.org/article/10.3847/2041-8213/aa8f41). The event was observed on 2017 August 17 at
12:41:06 UTC triggered by the Fermi gamma-ray Burst Monitor (GBM). T90 is 2.0   0.5 s, starting at T0 − 0.192 s. With these values, we set an approximate time window from **2017-08-17T12:41:05.80 UTC to 2017-08-17T12:41:07.80 UTC**.

In [ ]:
# Create a Space and Time MOC from the gravitational-wave sky localization
stmoc_gw170817 = STMOC.from_spatial_coverages(Time(['2017-08-17T12:41:03'], 
                                                   format="isot",  scale="utc"), 
                                              Time(['2017-08-17T12:41:09'], 
                                                   format="isot",  scale="utc"),
                                              [credible_region_final],time_depth=61)

# Create a Space and Time MOC from the GRB error boxes from Fermi/GBM
stmoc_grb170817 = STMOC.from_spatial_coverages(Time(['2017-08-17T12:41:05.80'], 
                                                    format="isot",  scale="utc"), 
                                               Time(['2017-08-17T12:41:07.80'], 
                                                    format="isot",  scale="utc"),
                                               [fermi_gbm], time_depth=61)

# Compute their space and time intersection and check that it is not empty
space_time_intersection_gw_grb = stmoc_grb170817.intersection(stmoc_gw170817)

if space_time_intersection_gw_grb.is_empty() != True:

    print("The time intersection between GW and GRB events ranges from:", 
          space_time_intersection_gw_grb.min_time.utc.iso +
          ' to ' + space_time_intersection_gw_grb.max_time.utc.iso)
    print()
    # Compute their space intersection 
    space_intersection_gw_grb = fermi_gbm.intersection(credible_region_final)
    area_int = round(space_intersection_gw_grb.sky_fraction*41253,2)
    
    print("The area intersection is: " + str(area_int) +'°^2' )
    print("The number of the disjoint regions: " + 
      str(space_intersection_gw_grb.split_count()))
    
    # Save in a FITS file
    space_intersection_gw_grb.write("space_intersection_gw_grb", format='fits',
                                  overwrite=True)
    
else:
    print("The two astrophysical events are not coincident")

## 6. Query Vizier catalogs
Here we query the galaxies collected in the [GLADE](https://ui.adsabs.harvard.edu/abs/2016yCat.7275....0D/abstract) catalog inside the intersection area between the GW170817 and GRB 170817. Then, we filter those galaxies according to the marginal distance posterior distribution integrated over the whole sky. The value reported in the header of GW170817 published in GWTC-1 [GW170814_skymap.fits.gz](https://dcc.ligo.org/public/0157/P1800381/007/GW170817_skymap.fits.gz).

In [ ]:
url_GW170814_skymap = "https://dcc.ligo.org/public/0157/P1800381/007/GW170817_skymap.fits.gz"
prob, header = hp.read_map(url_GW170814_skymap, h=True)

header = dict(header)
print(header)
print()

print('Posterior mean distance (Mpc):', + header['DISTMEAN'])
print('Posterior standard deviation of distance (Mpc):', + header['DISTSTD'])

In [ ]:
table = space_intersection_gw_grb.query_vizier_table('VII/281/glade2')
table

In [ ]:
# This is an expression for a 1-sigma cut on distance.
mask1 = table['Dist'] < header['DISTMEAN']+header['DISTSTD']
t1 = table[mask1]
mask2 = t1['Dist'] > header['DISTMEAN']-header['DISTSTD']
table_filtered = t1[mask2]
table_filtered

In [ ]:
from ipyaladin import Aladin
import ipyaladin as ipyal

aladin = ipyal.Aladin(target='NGC 4993', fov=180, survey ='P/DSS2/color')
aladin

In [ ]:
aladin.add_table(table_filtered)
aladin.add_moc_from_URL('space_intersection_gw_grb' ,{'color': 'gold' , 'opacity': 0.2,
                            'adaptativeDisplay': False, 
                                        'name': 'Intersection area between GW170817 and GRB 170817'})

**NOTE.** Mocpy does not yet implement a galaxy catalog query by the three-dimensional posterior probability
distribution. However, it is currently possible in Aladin to search for galaxies within the 2D credible region on the sky and, afterwards, apply a distance cut that is independent of sky position. Dedicated functionalities
are provided in the Python library [ligo.skymap](https://lscsoft.docs.ligo.org/ligo.skymap/#).

In [ ]:
aladin = Aladin(
    survey="http://alasky.cds.unistra.fr/DECaPS/DR1/color/",
    show_coo_grid=True,
    target="galactic center",
    coo_frame="galactic",
    fov=40,
    height=600
)
aladin